In [1]:
import os
import time
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
json_rpc_url = os.environ.get("JSON_RPC_POLYGON", "https://polygon-rpc.com")
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
Pair = ABI().get_contract(web3, "sushi/UniswapV2Pair.json")
filter = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
events = {}

evt: LogResult
for index, evt in enumerate(ReadEvents().apply(
            web3,
            start_block=start,
            end_block=end,
            filter=filter,
)):
    
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = 'polygon'
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['event_type'] = evt["event"]
    event['event_details'] = {}
    event['event_details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['event_details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['event_details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['event_details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[index] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,276,549 tx:0x4eae59c2ae99786a6b50b07b7e5112d7d85bb32859063ec984448712682fcb17
Swap at block:60,276,550 tx:0x0dccc717e99c571a1510fd237d34c78929b5a0bf86f6ff16486e53624febda45
Swap at block:60,276,550 tx:0xabc79c42200690cddb15d32fe541ecbe2fb3ee95b509494cba96eb512d95fb4f
Swap at block:60,276,552 tx:0x6d9e801b96f84c0677263116de1002cea8e4472a4b0a66f3f7b5c04f8a7be491
Swap at block:60,276,553 tx:0x37e2da03daf49e37794a02a19a4c75e888cef742ead59259b5159358b81eedb6
Swap at block:60,276,553 tx:0xc9c9b410e078dfd8b0ba0c2e1013d96109895d3519c86b04505517b37f4769e9
Swap at block:60,276,557 tx:0x2b4ba477215cda7a1486193de1372986533f540c882618a9349f046282fbef82
Swap at block:60,276,557 tx:0x2b4ba477215cda7a1486193de1372986533f540c882618a9349f046282fbef82
Swap at block:60,276,558 tx:0xea2833994e6a6743989996c82a6e6ed12b418cee31fc6cbb4de2c0ae9fc17fe0
.


In [8]:
events

{0: {'chain': 'polygon',
  'address': '0xbc0b623bd202b5ffd6c9aa403140b32304fdb9cf',
  'tx_hash': '0x4eae59c2ae99786a6b50b07b7e5112d7d85bb32859063ec984448712682fcb17',
  'blk_num': 60276549,
  'timestamp': 1722980450,
  'event_type': web3._utils.datatypes.Swap,
  'event_details': {'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'amount0In': 0,
   'amount1Out': 0}},
 1: {'chain': 'polygon',
  'address': '0x882df4b0fb50a229c3b4124eb18c759911485bfb',
  'tx_hash': '0x0dccc717e99c571a1510fd237d34c78929b5a0bf86f6ff16486e53624febda45',
  'blk_num': 60276550,
  'timestamp': 1722980452,
  'event_type': web3._utils.datatypes.Swap,
  'event_details': {'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0x07Ff4e06865de4934409Aa6eCea503b08Cc1C78d',
   'amount0In': 41962566527442920000,
   'amount1Out': 1475398416}},
 2: {'chain': 'polygon',
  'address': '0x1a6f6af2864b1f059a2e070140e373d6e3aaa2a1',
  'tx_hash': '